Once you have run the first notebook for the registration (**1_antspy_morphing_windows.ipynb**), we can proceed to the ANTsPy registration. <br>
You should be running this notebook in your Linux subsystem, and have already uploaded the files your generated in the first notebook.

You can find the source code for **ANTsPy** [here](https://github.com/ANTsX/ANTsPy).

In [1]:
# %matplotlib widget
%matplotlib notebook

In [2]:
import numpy as np
import flammkuchen as fl
from pathlib import Path
import tifffile
import ants
import os
import pandas as pd

import ants_tools as utilities

from skimage.exposure import match_histograms
import matplotlib.pyplot as plt
from ipywidgets import interact, fixed
from ipywidgets import widgets

from quickdisplay import *

## LOADING STACKS

In [3]:
path_list = list(Path(r'/home/otprat/velos_data').glob('*_f*'))
path_list

PosixPath('/home/otprat/velos_data/210722_f5')

In [ ]:
fish_path = path_list[0]
print('Working on {}'.format(fish_path.name))

In [4]:
registration_dir = fish_path #Path to where you uploaded the files generated in the windows notebook

#Load stacks
ref_stack = fl.load(registration_dir / "ref_mapped.h5")
mov_stack = fl.load(registration_dir / "mov_mapped.h5" ) 

We will perform first some histogram scaling and matching.<br>
This can generally make the registration process a bit easier, but one should inspect the stack to make sure that the brighter regions are similar between the two stacks.

In [5]:
#Scale brightness
ref_stack = to_255_range(ref_stack, quantile_max=0.9999)
mov_stack = to_255_range(mov_stack, quantile_min=0.01, quantile_max=0.9999)

#Histogram matching
ref_stack = match_histograms(ref_stack, mov_stack)

In [6]:
interact(plot_side_to_side,
        stack1=fixed(ref_stack),
        stack2=fixed(mov_stack),
        depth = widgets.IntSlider(min=0, max=100, step=5, continuous_update=False),
        dim = fixed(2),
        stack1_title=fixed('Ref stack'),
        stack2_title=fixed('Functional stack'))

interactive(children=(IntSlider(value=0, continuous_update=False, description='depth', step=5), Output()), _do…

<function quickdisplay.plot_side_to_side(stack1, stack2, depth, dim, stack1_title='Stack 1', stack2_title='Stack 2')>

## REGISTRATION

In [7]:
#Convert to AntsPy image format
ref_img = ants.from_numpy(ref_stack).clone("float")
mov_img = ants.from_numpy(mov_stack).clone("float")

Here we import an initial transformation matrix to help improve the result from the registration. If you don't have such matrix, go back to the Windows notebook, and run it to do some initial manual alignment. <br>
It is not essential, but definitely useful. If you want to skip this step, make sure to remove the `initial_transform` argument when calling the `ants.registration` function.

In [8]:
#Converting the initial transormation matrix (found with the manual annotations) to ANTs format
transform_mat = fl.load(registration_dir / "initial_transform_mapped.h5")
path_initial = str(registration_dir / "initial_transform_mapped.mat")
at = ants.create_ants_transform(transform_type='AffineTransform', precision='float', dimension=3,
                                matrix=transform_mat[:, :3], offset=transform_mat[:, 3])
ants.write_transform(at, path_initial)

Here we perform the registration. 

I generally like to perform first a nice affine transformation that morphs the two stacks as well as possible, and then play around with the parameters for the deformation. For a high quality affine mapping, the `'TRSAA'` transformation type is very suitable, and then the diffeomorphic transformation can be done via `'SyNOnly'` transforms that only introduce deformations. However, one can also apply both transformations simultaneously via a `'SyNRA'` transformation.  <br> Doing this process in a single or multiple steps will affect how the transformations need to be applied later on to coordinates in order to transform single point locations. For more information on that topic, check the tutorial [here](https://github.com/ANTsX/ANTsPy/blob/master/tutorials/concatenateRegistrations.ipynb).

Check the [documentation](https://antspy.readthedocs.io/en/latest/registration.html) about `ants.registration` for more information about transformation types and the arguments that they accept.

*Note on troubleshooting*: if the regsitration functions returns your reference stack as the mov_out image, it means there is an error in your call of the function. Check the linux terminal to try to find where the issue is.

### Affine registration

In [9]:
%%time
affine_tx = ants.registration(fixed=ref_img, 
                              moving=mov_img,
                              initial_transform=path_initial,
                              type_of_transform='TRSAA',
                              aff_metric='mattes',
                              aff_sampling=1,
                              total_sigma=0,
                              flow_sigma=0,
                              reg_iterations=[20, 15, 10],
                              outprefix=str(registration_dir / 'affine_tx_'),
                              verbose=True
                             )

CPU times: user 12min 47s, sys: 10.4 s, total: 12min 57s
Wall time: 5min 7s


As an intermediate step, we can retrieve the morphed image and see how the affine transform performs by itself

In [10]:
#Retrieve output image
affine_stack = affine_tx['warpedmovout'].numpy()
fl.save(registration_dir / '{}_affine.h5'.format(fish_path.name), affine_stack)

In [11]:
interact(plot_overlay,
         ref=fixed(ref_stack),
         mov=fixed(affine_stack),
         plane=widgets.IntSlider(min=0, max=ref_stack.shape[2]-1, step=1, continuous_update=False),
         dim = fixed(2),
         stack1_title=fixed('Reference'),
         stack2_title=fixed('Affine movout'))

interactive(children=(IntSlider(value=0, continuous_update=False, description='plane', max=128), Text(value='R…

<function quickdisplay.plot_overlay(ref, mov, plane, dim, ref_title='Reference', mov_title='Warped movout')>

### Diffeomorphic registration

And now we can proceed with the diffeomorphic transformation. This time, when calling the morphing function, we will use the `SyNOnly` method, which performs only a deformation transformation, and we will specify as the initial transformation, the resulting affine derived in the previous step. <br>
Moreover, especially with these defomative tranformations, we want to play around a lot with the other parameters that we are passing to the registration function.

In [ ]:
initial_affine_tx = str(next(registration_dir.glob('affine_tx*')))

In [16]:
%%time
warp_tx = ants.registration(fixed=ref_img,
                            moving=mov_img,
                            initial_transform=initial_affine_tx,
                            type_of_transform='SyNOnly',
                            syn_sampling=80, 
                            flow_sigma=10, 
                            reg_iterations=[15, 10, 5],
                            outprefix=str(registration_dir / 'warp_tx_'),
                            verbose=True
                           )

CPU times: user 25min 43s, sys: 1min 38s, total: 27min 21s
Wall time: 12min 7s


And we can finally retrieve the registered images and see how well they match the reference stack:

In [18]:
#Retrieve output image
warp_stack = warp_tx['warpedmovout'].numpy()
fl.save(registration_dir / '{}_warp.h5'.format(fish_path.name), warp_stack)

In [19]:
interact(plot_overlay,
         ref=fixed(ref_stack),
         mov=fixed(warp_stack),
         plane=widgets.IntSlider(min=0, max=ref_stack.shape[2]-1, step=1, continuous_update=False),
         dim = fixed(2),
         stack1_title=fixed('Reference'),
         stack2_title=fixed('Warped movout'))

interactive(children=(IntSlider(value=0, continuous_update=False, description='plane', max=128), Text(value='R…

<function quickdisplay.plot_overlay(ref, mov, plane, dim, ref_title='Reference', mov_title='Warped movout')>

Diffeomorphic transformations can be a bit tricky to get right, as in order to optimize the registration, minimization algorithms can sometimes perform very sharp or radical warpings on our stacks. While these can result in nice registrations, it can also happen that they pick up on small details or artifacts and the registrations become excessively deformed. This is why it is important to tinker with the parameters from the prvious registration function.


In order to see how these warp fields deform our stacks, we can look into the Jacobian of the deformations, to get an idea of how each pixel is affected by the deformations. <br>
The Jacobian determinant of our deformation matrix provides an idea of how the space around a pixel is deformed as a result of the warping. Values above 1 indicate that the area of neighboring pixels is being stretched, while for values below 1, these areas are being compressed. Below, we plot the log of this determinant.<br>
For a nice series on Jacobian matrices, you can check [these videos](https://www.khanacademy.org/math/multivariable-calculus/multivariable-derivatives/jacobian/v/jacobian-prerequisite-knowledge).

In [66]:
jacobian_det_mat_log = ants.create_jacobian_determinant_image(ref_img, warp_tx['fwdtransforms'][0], do_log=True)

In [68]:
interact(view_jacobian,
        ref=fixed(ref_stack),
        mov_affine=fixed(affine_stack),
        mov_warped=fixed(warp_stack),
        jacobian_mat=fixed(jacobian_det_mat_log),
        plane=widgets.IntSlider(min=0, max=ref_stack.shape[2]-1, step=1, continuous_update=False),
        dim = fixed(2))

interactive(children=(IntSlider(value=0, continuous_update=False, description='plane', max=128), Output()), _d…

<function __main__.view_jacobian(ref, mov_affine, mov_warped, jacobian_mat, plane, dim)>

## COORDINATE TRANSFORMATION

And finally, we can import the coordinates for our ROIs and apply those same transformations to them.

When applying ANTsPy transformations (whether it is to point coordinates or stacks) one needs to pay attention to the arguments `transformlist` and `whichtoinvert`.

- The inputs for `transformlist` you have to pass for each one will depend on the order in which you performed your registrations, as well as on whether your specific registrations are performing different types of registrations simultaneously (i.e., affine + diffeomorphic).

- The same goes for the `whichtoinvert` one, which basically points to which of the provided transformations in `transformlist` need to be inverted before being applied (inversion is required for affine transformations, but not for deformations).

For more information on how to concatenate transformations, you can check this [tutorial](https://github.com/ANTsX/ANTsPy/blob/master/tutorials/concatenateRegistrations.ipynb).

In [19]:
#Pointer strings to transforms
affine_path = next(registration_dir.glob('warp_tx*Affine*'))
warp_path = next(registration_dir.glob('warp_tx*InverseWarp*'))

#Prepare antspy transformation function inputs
tx_list = list([str(affine_path), str(warp_path)])
inversion_list = [True, False]

#Load ROI coordinates
mov_coords = fl.load(registration_dir / "mov_roi_coords_mapped.h5")

#Create DataFrame with coordinates to transform
mov_coords_df = pd.DataFrame(mov_coords, columns=['x', 'y', 'z'])

#Transform ROI coordinates
mov_coords_trans = ants.apply_transforms_to_points(3, mov_coords_df, transformlist=tx_list, whichtoinvert=inversion_list).values

#Store
fl.save(fish_path / 'mov_roi_coords_transformed.h5', mov_coords_trans)

In [ ]:
#A check to make sure all ROIs are morphed inside the reference brain
plt.figure()

plt.imshow(np.nanmean(ref_stack, 2), cmap='gray')
plt.scatter(mov_coords_trans[:, 1], mov_coords_trans[:, 0], c='red', alpha=.075)